# Module 08: Agentic RAG
This notebook demonstrates how traditional Chroma DB works for RAG pipelines.

## What we'll learn:
- ChromaDB
- OpenAI Embeddings
- RAG using State Machine
- Retrieval, Augment and Generation as steps

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
import os
import chromadb
from chromadb.utils import embedding_functions
from chromadb.api.models.Collection import Collection
import pdfplumber
from dotenv import load_dotenv
from typing import TypedDict, List

from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Resource
from lib.llm import LLM
from lib.messages import BaseMessage, UserMessage, SystemMessage

In [ ]:
import logging
logging.getLogger('pdfminer').setLevel(logging.ERROR)

In [ ]:
sentence_list = [
    "Meta drops multimodal Llama 3.2 — here's why it's such a big deal",
    "Chip giant Nvidia acquires OctoAI, a Seattle startup that helps companies run AI models",
    "Google is bringing Gemini to all older Pixel Buds",
    "The first Intel Battlmage GPU benchmarks have leaked",
    "Dell partners with Nvidia to accelerate AI adoption in telecoms",
]
ids = ["id1", "id2", "id3", "id4", "id5"]

## ChromaDB with Default Embedding Function

In [ ]:
chroma_client = chromadb.Client()

In [ ]:
collection = chroma_client.create_collection(
    name="demo"
)

In [ ]:
collection.add(
    documents=sentence_list,
    ids=ids
)

In [ ]:
collection.count()

In [ ]:
collection.peek(1)

In [ ]:
collection.query(
    query_texts=["gadget"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

In [ ]:
result = collection.query(
    query_texts=["gadget"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

result['documents'][0]

In [ ]:
print(collection._embedding_function.name())

In [ ]:
size = len(collection.peek(1)['embeddings'][0])
print(f"Size of the embeddings array: {size}")


## OpenAI Embeddings

In [ ]:
chroma_client.delete_collection(name="demo")

In [ ]:
embeddings_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [ ]:
collection = chroma_client.create_collection(
    name="demo",
    embedding_function=embeddings_fn
)

In [ ]:
collection.add(
    documents=sentence_list,
    ids=ids
)

In [ ]:
collection.query(
    query_texts=["gadget"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

In [ ]:
print(collection._embedding_function.name())

In [ ]:
size = len(collection.peek(1)['embeddings'][0])
print(f"Size of the embeddings array: {size}")

## RAG

**Load**

In [ ]:
file_path = "GlobalEVOutlook2025.pdf"
documents = []
page_nums = []

In [ ]:
with pdfplumber.open(file_path) as pdf:
    for num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()
        if text:
            documents.append(text)
            page_nums.append(str(num))


In [ ]:
collection = chroma_client.create_collection(
    name="traditional_rag",
    embedding_function=embeddings_fn
)

In [ ]:
collection.add(
    documents=documents,
    ids=page_nums
)

**State Machine**

In [ ]:
class State(TypedDict):
    messages: List[BaseMessage]
    question: str
    documents: List[str]
    answer: str

**RAG: Retrieve**

In [ ]:
def retrieve(state:State, resource:Resource):
    question = state["question"]
    collection:Collection = resource.vars.get("collection")
    results = collection.query(
        query_texts=[question],
        n_results=3,
        include=['documents']
    )
    retrieved_docs = results['documents'][0]

    return {"documents": retrieved_docs}

**RAG: Augment**

In [ ]:
def augment(state:State):
    question = state["question"]
    documents = state["documents"]
    context = "\n\n".join(documents)

    messages = [
        SystemMessage(content="You are an assistant for question-answering tasks."),
        UserMessage(
            content=(
                "Use the following pieces of retrieved context to answer the question. "
                "If you don't know the answer, just say that you don't know. "
                f"\n# Question: \n-> {question} "
                f"\n# Context: \n-> {context} "
                "\n# Answer: "
            )
        )
    ]

    return {"messages": messages}

**RAG: Generate**

In [ ]:
def generate(state:State, resource:Resource):
    llm:LLM = resource.vars.get("llm")
    ai_message = llm.invoke(state["messages"])
    return {
        "answer": ai_message.content,
        "messages": state["messages"] + [ai_message],
    }

In [ ]:
workflow = StateMachine(State)

In [ ]:
# Create steps
entry = EntryPoint()
retrieve_step = Step("retrieve", retrieve)
augment_step = Step("augment", augment)
generate_step = Step("generate", generate)
termination = Termination()

workflow.add_steps(
    [
        entry,
        retrieve_step,
        augment_step,
        generate_step,
        termination
    ]
)

In [ ]:
# Add transitions
workflow.connect(entry, retrieve_step)
workflow.connect(retrieve_step, augment_step)
workflow.connect(augment_step, generate_step)
workflow.connect(generate_step, termination)

In [ ]:
llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

In [ ]:
resource = Resource(
    vars = {
        "llm": llm,
        "collection": collection,
    }
)

In [ ]:
initial_state: State = {
    "question": "What was the number of electric car sales and their market share in Brazil in 2024?",
}

In [ ]:
run_object = workflow.run(initial_state, resource)

In [ ]:
run_object.get_final_state()["answer"]